In [12]:
import scanpy as sc
import numpy as np
from scipy.spatial import cKDTree
from PIL import Image
import tifffile as tiff

def compute_distance_to_plaques(adata, image_path, rotation_angle=0, flip_vertically=False):
    """
    Compute the distance of each cell in adata to the plaque in the segmented plaque image

    params:
    - adata
    - image_path: path to plaque mask
    - rotation_angle: angle by which to rotate image
    - flip_vertically: if needed to be flipped (default: false) 
    """
    
    # Load  TIFF file
    image = tiff.imread(image_path)
    image_pil = Image.fromarray(image)
    
    # Rotate image by  specified angle
    image_rotated = image_pil.rotate(rotation_angle, expand=True)

    # Optionally flip the image vertically
    if flip_vertically:
        image_rotated = image_rotated.transpose(Image.FLIP_TOP_BOTTOM)

    # Convert image to a numpy array , find coordinates of plaque regions
    mask_array = np.array(image_rotated)
    foreground_coords = np.column_stack(np.where(mask_array > 0))

    # Create spatial index of plaque coordinates
    tree = cKDTree(foreground_coords)

    # Get cell coordinates fro AnnData object and get distances, store in adata
    cell_coords = np.column_stack([adata.obs['y'], adata.obs['x']])
    distances, _ = tree.query(cell_coords)
    adata.obs['distance_to_plaques'] = distances

    return adata

# dfine file paths and rotation parameters 
datasets = [
    ('B1S3', '../100.Segmentation/B1S3/B1S3_CRX_plaque_polygon_mask.tif', -90, False),
    ('B1S4', '../100.Segmentation/B1S4/B1S4_CRX_plaque_polygon_mask.tif', 90, False),
    ('B2S3', '../100.Segmentation/B2S3/B2S3_CRX_plaque_polygon_mask.tif', -90, False),
    ('B2S4', '../100.Segmentation/B2S4/B2S4_CRX_plaque_polygon_mask.tif', 90, True)
]

# Process each dataset
for dataset_name, image_path, rotation_angle, flip_vertically in datasets:
    adata = sc.read_h5ad(f'../110.Alignment_ROIs/40um_{dataset_name}_crx.h5ad')

    adata = compute_distance_to_plaques(adata, image_path, rotation_angle, flip_vertically)

    adata.write_h5ad(f'../{dataset_name}_CRX_40um_plaqueDistances.h5ad')


### concatenate all the results into one adata 

In [ ]:
B1S3 = sc.read_h5ad('../B1S3_CRX_40um_plaqueDistances.h5ad')
B1S4 = sc.read_h5ad('../B1S4_CRX_40um_plaqueDistances.h5ad')
B2S3 = sc.read_h5ad('../B2S3_CRX_40um_plaqueDistances.h5ad')
B2S4 = sc.read_h5ad('../B2S4_CRX_40um_plaqueDistances.h5ad')


In [ ]:

objects = {
    'B1S3': B1S3,
    'B1S4': B1S4,
    'B2S3': B2S3,
    'B2S4': B2S4
}



In [ ]:
combined_adata = sc.concat([B1S3, B1S4, B2S3, B2S4])

sc.pp.calculate_qc_metrics(combined_adata, qc_vars=["human", "mouse"], inplace=True)

combined_adata.write_h5ad('./combined_CRX_40um_plaqueDistances.h5ad')

In [ ]:
combined_adata_sub = combined_adata[combined_adata.obs['total_counts'] > 30]
combined_adata_sub = combined_adata_sub[combined_adata_sub.obs['total_counts'] < 250]

In [ ]:
combined_adata_sub.write_h5ad('./combined_CRX_40um_plaqueDistances_forDE.h5ad')